# RAG

In [31]:
import os
import re

from dotenv import load_dotenv
from typing import Dict, List

from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents.base import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

In [6]:
load_dotenv(dotenv_path='../src/.env')

True

## Functions

In [22]:
def load_urls(urls_path: str) -> List[str]:
    with open(urls_path, "r", encoding="utf-8") as file:
        urls = [line.strip() for line in file if line.strip()]
    return urls


def load_documents(urls_list: List[str]) -> List[Document]:
    docs = [WebBaseLoader(url).load() for url in urls_list]
    docs_list = [item for sublist in docs for item in sublist]
    print(f"len of documents :{len(docs_list)}")
    return docs_list


def preprocess_documents(documents: List[Document]) -> List[Document]:
    for doc in documents:
        doc.page_content = re.sub(r'\n+', '\n', doc.page_content.strip())
    return documents


def split_documents(documents: List[Document], chunk_size: int, chunk_overlap: int) -> List[Document]:
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    doc_splits = text_splitter.split_documents(documents)
    print(f"length of document chunks generated :{len(doc_splits)}")
    return doc_splits


In [11]:
urls = load_urls('../data/urls.txt')

In [14]:
documents = load_documents(urls)

len of documents :8


In [18]:
preprocessed_documents = preprocess_documents(documents)

In [26]:
splitted_documents = split_documents(preprocessed_documents, chunk_size=1024, chunk_overlap=200)

length of document chunks generated :156


In [32]:
embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:05<00:00,  1.08s/it]


In [33]:
vectorstore = Chroma.from_documents(documents=splitted_documents,
                                    embedding=embedding_model,
                                    collection_name="local-rag")

RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma                     requires sqlite3 >= 3.35.0.[0m
[94mPlease visit                     https://docs.trychroma.com/troubleshooting#sqlite to learn how                     to upgrade.[0m